# New York Times. Extract Named Entity Recognition

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import pymysql
import time
import seaborn as sns
import spacy

Analyse titles from public available data through NYT API. 
Titles are stored in MySQL database in a table NYT_ARTICLES.

In [2]:
#connect to MySQL database

conn = pymysql.connect(db="db",
                       user="user",
                       passwd="password",
                       host= "host", 
                       port = 3306
                       )
print('Connected to MySQL.')

cursor = conn.cursor()

Connected to MySQL.


In [21]:
sql_query_select = "SELECT title FROM DT_NYT.V_NYT_ARTICLE_INFO_7;"
sql_select_id = "select id_info from NYT_ARTICLE_INFO_7"
sql_insert = "INSERT INTO NYT_ARTICLE_INFO_7_NER (id_info, 'PERSON', 'GPE', 'NORP', 'FAC', 'ORG', 'LOC', 'PRODUCT', 'EVENT', 'WORK_OF_ART') VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

In [8]:
df = pd.read_sql(sql_query_select, conn)

In [6]:
nlp = spacy.load('en', entity = False, parser = False)
entities = ['PERSON', 'GPE', 'NORP', 'FAC', 'ORG', 'LOC', 'PRODUCT', 'EVENT', 'WORK_OF_ART']

#df_q2['GoldParse'] = df.output_ne.apply(lambda x: GoldParse(nlp.make_doc(x), entities).orig_annot)

In [11]:
t = time.time()

#transform articles in Spacy nlp format
df['title_nlp'] = df.title.apply(lambda x: nlp(x))

print('Running time: {} mins'.format(round((time.time() - t) / 60, 2)))

Running time: 267.72 mins


In [12]:
#extract country, city, person name most probable in the articles per event
df['NER'] = df.title_nlp.apply(lambda x: '; '.join(item for item in set([ent.text for ent in x.ents 
                                                                 if ent.label_ in entities])))

In [18]:
df['PERSON'] = df.title_nlp.apply(lambda x: '; '.join(item for item in set([ent.text for ent in x.ents 
                                                                 if ent.label_== entities[0]])))
df['GPE'] = df.title_nlp.apply(lambda x: '; '.join(item for item in set([ent.text for ent in x.ents 
                                                                 if ent.label_== entities[1]])))
df['NOPR'] = df.title_nlp.apply(lambda x: '; '.join(item for item in set([ent.text for ent in x.ents 
                                                                 if ent.label_== entities[2]])))
df['FAC'] = df.title_nlp.apply(lambda x: '; '.join(item for item in set([ent.text for ent in x.ents 
                                                                 if ent.label_== entities[3]])))
df['ORG'] = df.title_nlp.apply(lambda x: '; '.join(item for item in set([ent.text for ent in x.ents 
                                                                 if ent.label_== entities[4]])))
df['LOC'] = df.title_nlp.apply(lambda x: '; '.join(item for item in set([ent.text for ent in x.ents 
                                                                 if ent.label_== entities[5]])))
df['PRODUCT'] = df.title_nlp.apply(lambda x: '; '.join(item for item in set([ent.text for ent in x.ents 
                                                                 if ent.label_== entities[6]])))
df['EVENT'] = df.title_nlp.apply(lambda x: '; '.join(item for item in set([ent.text for ent in x.ents 
                                                                 if ent.label_== entities[7]])))
df['WORK_OF_ART'] = df.title_nlp.apply(lambda x: '; '.join(item for item in set([ent.text for ent in x.ents 
                                                                 if ent.label_== entities[8]])))

In [22]:
df_id = pd.read_sql(sql_select_id, conn)

In [24]:
df.drop('title', axis = 1 , inplace = True)
df.drop('title_nlp', axis = 1 , inplace = True)
df.drop('NER', axis = 1 , inplace = True)

In [25]:
df0 = pd.concat([df_id, df], axis = 1)
df0.to_csv('NER.csv')